#  Create Pseudo-Parallel Dataset with Style Scores

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
    
import sys, os
sys.path.append('../paraphrase/')
sys.path.append('../jointclassifier/')
from paraphraser_args import ModelArguments as pma, DataTrainingArguments as pda, TrainingArguments as pta
from paraphraser_dataloader import load_dataset as pld, load_dataset_style as lds
from paraphraser_trainer import ParaphraserTrainer
from transformers import AutoConfig, AutoTokenizer, AutoModelWithLMHead, HfArgumentParser
from joint_args import ModelArguments as jma, DataTrainingArguments as jda, TrainingArguments as jta
from joint_dataloader import load_dataset as jld
from joint_trainer import JointTrainer
from joint_model_v1 import JointSeqClassifier

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from tqdm.notebook import tqdm, trange
from torch import cuda, no_grad

## Load in desired dataset and paraphraser model
In the cell below, define the dataset you want to work with and the paraphraser model (here a `"t5-small"` [from Hugging Face](https://huggingface.co/t5-small))

In [47]:
model_name = "t5_paraphrase"
paraphrase_task = 'formality'
model_nick = "t5_paraphrase"
model_type = 't5-small'
output_dir = "../models/"
epochs = "3"
train_batch_size = "16"
eval_batch_size = "16"
save_log_steps = "400"

parser = HfArgumentParser((pma, pda, pta))
model_args_para, data_args_para, training_args_para = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    model_name,
    "--model_nick",
    model_nick,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, model_nick),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    save_log_steps,
    "--save_steps",
    save_log_steps,
    "--data_parallel",
    "True"
])


PyTorch: setting up devices


In [5]:
# Create the paraphraser tokenizer and dataset objects
para_tokenizer = AutoTokenizer.from_pretrained(model_type, cache_dir=model_args_para.cache_dir,
                                         model_max_length = data_args_para.max_seq_len)
dataset = lds(data_args_joint.data_dir, para_tokenizer,
                            task=paraphrase_task, mode="train", n_proc=6000)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at ../models/cache/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
  

In [6]:
# Use the paraphrase configuration defined above to create the model
model = AutoModelWithLMHead.from_pretrained(os.path.join(output_dir, model_name))
#training_args_para.output_dir)

/home/dmac/Documents/MIMS Coursework/Capstone/env_marvin/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:966: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file ../models/t5_paraphrase/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relat

## Use the Paraphraser to Generate Predictions

In [12]:
sampler = SequentialSampler(dataset)
dataloader = DataLoader(dataset, sampler=sampler, batch_size=16)

num_return_sequences = 3

device = ("cuda" if cuda.is_available() else "cpu") #and not self.args.no_cuda
model = model.to(device)
model.eval()
predicted1 = []
predicted2 = []
predicted3 = []

epoch_iterator = tqdm(dataloader, desc="Iteration")
with no_grad():
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)  # GPU or CPU
        generated_outputs = model.generate(input_ids = batch[0], 
                                           attention_mask = batch[1], 
                                           max_length=70, 
                                           num_beams=9,
                                           early_stopping=True,
                                           encoder_no_repeat_ngram_size=5,
                                           num_beam_groups=3,
                                           diversity_penalty=0.5,
                                           num_return_sequences=num_return_sequences)
        paras = para_tokenizer.batch_decode(generated_outputs.detach().cpu().numpy(), 
                                                 skip_special_tokens=True)
        predicted1 += paras[0::3]
        predicted2 += paras[1::3]
        predicted3 += paras[2::3]

## Save results to a csv file

In [13]:
import pandas as pd

In [14]:
data_dir = "../data/processed_filtered/"

In [16]:
# Store outputs to disk using in_filename as the original texts 
# and writing outputs to out_filename

# If you want to do other parts of the dataset other than train, 
# set the mode in 'dataset' above to the desired mode and then rerun the paraphrase
# and change these filenames to point to the slice of the data you want to use (dev, test, etc.)
in_filename = 'train.csv'
out_filename = 'train_paraphrased.csv'

df_para = pd.DataFrame(data={'paraphrased1' : predicted1, 
                             'paraphrased2' : predicted2, 
                             'paraphrased3' : predicted3}) 
df = pd.read_csv(os.path.join(data_dir, task, in_filename), names =['text', 'label'])
df['paraphrased1'] = df_para['paraphrased1']
df['paraphrased2'] = df_para['paraphrased2']
df['paraphrased3'] = df_para['paraphrased3']
df.to_csv(os.path.join(data_dir, task, out_filename), 
               header=False, index=False)

In [17]:
# Inspect some results
df.head()

,text,label,paraphrased1,paraphrased2,paraphrased3
0,uhh nobody cause hes butttttt ugly can you rea...,0,you can't read it or ru a stupid like him.,you can't read it or ru a stupid man like him.,you can't read it or ru a stupid man as he is.
1,but dont force it on her,0,but don't force her to force her.,but don't force her to force her to force her.,but she won't force her to force her.
2,im 8 and I like tom felton and he is 18 how sh...,0,I'm 8 and I like him and he's 18 and I'm 8 and...,I'm 8 and he's 18 and I like him and he's 18 a...,I'm 8 and he's 18 and I like him and he's 18 h...
3,I can't even visit her or nothin and she lives...,0,"I'm not going to visit her, and she's upstairs!","I'm not going to visit her, and she lives upst...","I'm not going to visit her, and she's upstairs."
4,"Sadly, the man was from Spain and not from Mex...",1,"unfortunately, the man came from Spain and not...","unfortunately, he's not from Mexico, but from ...","unfortunately he was from Spain, not Mexico."


In [ ]:
df.tail()

## Now use classifier for Scoring
This may cause GPU memory issues, so it's possible you may have to shutdown the kernel and restart without running the paraphraser first to run this next portion. If doing so, reload the df that was written to disk in several cells above.  

## Load in desired dataset and classifier model
In the cell below, define the dataset you want to work with and the classifier model.

In [32]:
task = "formality+emo"
data_dir = "../data/processed_filtered/"
model_name = "distilbert-base-uncased"
model_nick = "distilbert_uncased_2"
output_dir = "../models/"
freeze_encoder = "False"
skip_preclassifier = "False"
train_jointly = "True"
epochs = "5"
train_batch_size = "256"
eval_batch_size = "512"
log_save_steps = "200"

parser = HfArgumentParser((jma, jda, jta))
model_args_joint, data_args_joint, training_args_joint = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    model_name,
    "--model_nick",
    model_nick,
    "--task",
    task,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, model_nick, task, 'joint'),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--freeze_encoder",
    freeze_encoder,
    "--skip_preclassifier",
    skip_preclassifier,
    "--train_jointly",
    train_jointly,
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    log_save_steps,
    "--save_steps",
    log_save_steps
])


PyTorch: setting up devices


In [33]:
model_config = AutoConfig.from_pretrained(model_args_joint.model_name_or_path, 
                                          cache_dir=model_args_joint.cache_dir)
tokenizer = AutoTokenizer.from_pretrained(model_args_joint.model_name_or_path, 
                                          cache_dir=model_args_joint.cache_dir,
                                          model_max_length = data_args_joint.max_seq_len)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at ../models/cache/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.3.3",
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at ../models/cache/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd

In [34]:
# Load data as expected by joint classifier
tasks = data_args_joint.task.split('+')
train_dataset, idx_to_classes = jld(data_args_joint.data_dir, 
                                             tokenizer, 
                                             model_name=model_args_joint.model_name_or_path, 
                           tasks=tasks, mode="train", n_proc=6000)
dev_dataset, _ = jld(data_args_joint.data_dir, 
                              tokenizer, 
                              model_name=model_args_joint.model_name_or_path, 
                              tasks=tasks, mode="dev", n_proc=6000)

 33%|███▎      | 1/3 [00:00<00:00,  8.94it/s]

torch.Size([50000, 64]) torch.Size([50000, 64]) torch.Size([50000, 2]) torch.Size([50000])


 33%|███▎      | 1/3 [00:00<00:00,  6.82it/s]

torch.Size([62708, 64]) torch.Size([62708, 64]) torch.Size([62708, 2]) torch.Size([62708])


100%|██████████| 1/1 [00:00<00:00,  6.55it/s]

torch.Size([12500, 64]) torch.Size([12500, 64]) torch.Size([12500, 2]) torch.Size([12500])
torch.Size([15730, 64]) torch.Size([15730, 64]) torch.Size([15730, 2]) torch.Size([15730])


In [35]:
label_dims = {task : 1 if len(list(idx_to_classes[task].keys())) == 2 else len(list(idx_to_classes[task].keys())) for task in idx_to_classes}
label_dims

{'formality': 1, 'emo': 1}

In [40]:
joint_model = JointSeqClassifier.from_pretrained(os.path.join(output_dir,
                                                              model_args_joint.model_nick,task),
                                           tasks=tasks,
                                           model_args=model_args_joint,
                                           task_if_single=None, 
                                           joint = training_args_joint.train_jointly,
                                           label_dims=label_dims)

trainer = JointTrainer([training_args_joint,model_args_joint, data_args_joint], 
                       joint_model, train_dataset, dev_dataset, idx_to_classes)

loading configuration file ../models/distilbert_uncased_2/formality+emo/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "JointSeqClassifier"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.3.3",
  "vocab_size": 30522
}

loading weights file ../models/distilbert_uncased_2/formality+emo/pytorch_model.bin
All model checkpoint weights were used when initializing JointSeqClassifier.

All the weights of JointSeqClassifier were initialized from the model checkpoint at ../models/distilbert_uncased_2/formality+emo.
If your task is similar to the task the model of the checkpoint was train

## Run classifier on paraphrased and original text

This is currently done with pd DataFrames but could probably be made better by using a batch data loader. 

In [41]:
import scipy.stats as ss
from tqdm import tqdm
tqdm.pandas()

In [42]:
tasks

['formality', 'emo']

In [43]:
def pred_paraphrases(row, tasks, cols):
    '''
    Make style predictions on a given df row for a given set of text columns
    and classification tasks. 
    '''
    preds = {}
    for col in cols:
        sentence = row[col]
        out = trainer.predict_for_sentence(sentence, tokenizer)
        for task in tasks:
            pred = float(out[task]['prob'])
            preds[task + '_' + col] = pred
    return preds

def get_best_pred(row, cols, target_val=0.5):
    '''
    Helper funtion for determiningg which paraphrase is 'best' 
    for a given set of paraphrase column style scores and a target value
    that you want the scores to be close to. Currently just outputs the best score
    but could be modified to get best sentence as well.
    '''
    best_diff = 1
    best_val = None
    for col in cols:
        diff = abs(row[col] - target_val)
        if diff < best_diff:
            best_val = row[col]
            best_diff = diff
    return best_val

In [44]:
# Define columns on which to run the classification
cols_to_use = ['text','paraphrased1', 'paraphrased2', 'paraphrased3']
# Define the names of the columns where the output scores will be stored
cols_preds = ['pred_formality_orig', 'pred_emo_orig',
              'pred_formality_paraphrased1', 'pred_emo_paraphrased1',
              'pred_formality_paraphrased2', 'pred_emo_paraphrased2',
              'pred_formality_paraphrased3', 'pred_emo_paraphrased3']
# Store results into df
df[cols_preds] = df.progress_apply(lambda x : pred_paraphrases(x, tasks, cols_to_use), 
                                   axis=1, result_type="expand")

100%|██████████| 50000/50000 [25:46<00:00, 32.33it/s]


In [48]:
# Store results of style classification:
out_filename = task + '_train_cross_predict_paraphrases.csv'

df.to_csv(os.path.join(data_dir, paraphrase_task, out_filename), header=True, index=False)